# Switching Assistant
- Intent classification and Service Routing
- The LLM analyzes this combination and classifies the input into one or combination of three main intents: AppAssistant, PainAssistant, or PersonalAssistant.
- Creating through bedrock-agent for more confirugation and control

In [1]:
import boto3
import sys
import json
import pprint
from botocore.client import Config
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
import os
import random
from retrying import retry
import time
from utility import *
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.35.34


In [2]:
# Bedrock Agent runtime
pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = 'ca-central-1'
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config, region_name = region)

# KB ID
kb_id = 'V2W0LT3GZP'